# Tesla Ticker| Histroical Stock Price Dataset| GRU| BiGRU

In [1]:
# Import Libararies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# implort plotly for interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
#import plot_acf
from statsmodels.graphics.tsaplots import plot_acf
# import seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose
# scaler
from sklearn.preprocessing import MinMaxScaler
# train_test_split
from sklearn.model_selection import train_test_split
# sequential using keras
from tensorflow.keras.models import Sequential
# Dense, Bidirectional,GRU Layers using keras
from tensorflow.keras.layers import Dense, Bidirectional, GRU
# Adam optimizer
from tensorflow.keras.optimizers import Adam
# Early stoppng or callbacks
from tensorflow.keras.callbacks import EarlyStopping
# import metrics
from sklearn.metrics import r2_score, mean_squared_error
# import warnings
import warnings 
warnings.filterwarnings('ignore')

2024-06-20 17:33:29.010197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:33:29.010414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:33:29.195465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data Exploration

In [2]:
df=pd.read_csv("/kaggle/input/tesla-stock-insights-and-predictions/tesla_stock_data.csv")
df.head()

,Date,Year,Open,High,Low,Close,Volume,Adj Close
0,2020-01-02,2020,28.299999,28.713333,28.114000,28.684000,142981500,28.684000
1,2020-01-03,2020,29.366667,30.266666,29.128000,29.534000,266677500,29.534000
2,2020-01-06,2020,29.364668,30.104000,29.333332,30.102667,151995000,30.102667
3,2020-01-07,2020,30.760000,31.441999,30.224001,31.270666,268231500,31.270666
4,2020-01-08,2020,31.580000,33.232666,31.215334,32.809334,467164500,32.809334


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       996 non-null    object 
 1   Year       996 non-null    int64  
 2   Open       996 non-null    float64
 3   High       996 non-null    float64
 4   Low        996 non-null    float64
 5   Close      996 non-null    float64
 6   Volume     996 non-null    int64  
 7   Adj Close  996 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 62.4+ KB


In [4]:
df.describe()

,Year,Open,High,Low,Close,Volume,Adj Close
count,996.000000,996.000000,996.000000,996.000000,996.000000,9.960000e+02,996.000000
mean,2021.493976,209.203720,213.956547,204.062843,209.121335,1.335149e+08,209.121335
std,1.118579,85.884404,87.588506,83.894602,85.708640,8.888939e+07,85.708640
min,2020.000000,24.980000,26.990667,23.367332,24.081333,2.940180e+07,24.081333
25%,2020.000000,159.780838,162.617496,154.389996,160.250004,7.718922e+07,160.250004
50%,2021.000000,223.989998,229.250000,218.266663,223.651665,1.074158e+08,223.651665
75%,2022.000000,263.002510,268.010002,258.120010,262.667496,1.578296e+08,262.667496
max,2023.000000,411.470001,414.496674,405.666656,409.970001,9.140820e+08,409.970001


In [5]:
df.columns

Index(['Date', 'Year', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [6]:
# Lets convert the Date column to datetime format
df['Date']=pd.to_datetime(df['Date'])
# Lets find the starting and ending date of the dataset
starting_date = df['Date'].min()
ending_date   = df['Date'].max()
# Lets check the Format of dates
starting_date_formatted = starting_date.strftime("%Y-%m-%d")
ending_date_formatted = ending_date.strftime("%Y-%m-%d")
# print the starting and ending date
print("Starting date is:", starting_date_formatted)
print("Ending date is:", ending_date_formatted)

Starting date is: 2020-01-02
Ending date is: 2023-12-29


In [7]:
print(f"Total days in this dataset which represents the stock price are: {(df.Date.max()-df.Date.min()).days}days")

Total days in this dataset which represents the stock price are: 1457days


In [8]:
df.isnull().sum().sort_values(ascending=True)

Date         0
Year         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
dtype: int64

## Fan Chart

In [14]:
# forcasted Closing price
forecasted_prices = np.random.normal(df['Adj Close'].mean(), df['Adj Close'].std(), len(df))

# Create upper and lower bounds for the fan chart
upper_bound = forecasted_prices + np.random.normal(0, 10, len(df))
lower_bound = forecasted_prices - np.random.normal(0, 10, len(df))

trace = go.Scatter(x=df['Date'], y=forecasted_prices, mode='lines', line=dict(color='purple'), name='Forecasted Prices')
trace_upper = go.Scatter(x=df['Date'], y=upper_bound, mode='lines', line=dict(color='lightgreen'), fill='tonexty', name='Upper Bound')
trace_lower = go.Scatter(x=df['Date'], y=lower_bound, mode='lines', line=dict(color='lightpink'), fill='tonexty', name='Lower Bound')

layout = go.Layout(title='Fan Chart of Forecasted Tesla Stock Prices', xaxis=dict(title='Date'), yaxis=dict(title='Price'))
fig = go.Figure(data=[trace, trace_upper, trace_lower], layout=layout)
iplot(fig)

## Moving Average Plot

In [31]:
df['Moving Average']=df['Close'].rolling(window=30).mean()
trace1=go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Closing price')
trace2=go.Scatter(x=df['Date'], y=df['Moving Average'], mode='lines', name='Moving Average')
layout=go.Layout(title='Stock Closing price with Moving average', xaxis=dict(title='Date'), yaxis=dict(title='Moving Average'))
fig=go.Figure(data=[trace1,trace2], layout=layout)
iplot(fig)

## Open Chart

In [30]:
rolling_mean=df['Open'].rolling(window=30).mean()
trace=go.Scatter(x=df['Date'], y=rolling_mean, mode='lines', fill='tozeroy', name='Rolling Mean')
layout=go.Layout(title='Area Chart Rolling Mean of Opening Tesla Stock Price', xaxis=dict(title='Opening Price'), yaxis=dict(title='Rolling Mean'))
fig=go.Figure(data=[trace], layout=layout)
iplot(fig)

## Volume VS Scatter Plot

In [29]:
trace = go.Scatter(x=df['Volume'], y=df['Close'], mode='markers',
                   marker=dict(color='purple', size=8), name='Volume vs. Closing Price')

layout = go.Layout(title='Volume vs. Closing Price Scatter Plot',
                   xaxis=dict(title='Volume'), yaxis=dict(title='Closing Price'))
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Seasonal Decomposition Plot

In [28]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(df['Close'], model='additive', period=30)

# Plot decomposition components
trace_trend = go.Scatter(x=df['Date'], y=decomposition.trend, mode='lines', name='Trend')
trace_seasonal = go.Scatter(x=df['Date'], y=decomposition.seasonal, mode='lines', name='Seasonal')
trace_residual = go.Scatter(x=df['Date'], y=decomposition.resid, mode='lines', name='Residual')

layout = go.Layout(title='Seasonal Decomposition of Tesla Stock Closing Prices', xaxis=dict(title='Date'), yaxis=dict(title='Price'))
fig = go.Figure(data=[trace_trend, trace_seasonal, trace_residual], layout=layout)
iplot(fig)